# **FROM MAGNETIC RELAXATION $\rightarrow$ (NON-AXISYMMETRIC) GRAD–SHAFRANOV**

If my understanding's correct, the Grad–Shafranov equation can be derived from the *"static force balance"* assumption
$$
\mathbf{j} \times \mathbf{B} = \nabla p,
$$
and assuming axisymmetry.
If this holds true, then it's necessarily true that
$$
\text{curl}[\mathbf{j} \times \mathbf{B}] = \mathbf{0}.
$$
This looks very similar to the relation $\mathbf{j} \times \mathbf{B} = \mathbf{0}$ we know to hold for equilibria in magnetic relaxation.
In fact, assuming a constant pressure on the boundary *(which would be a natural boundary condition, since the pressure is constant on flux surfaces)* the two are equivalent on toroidal domains *(such as those in a tokamak/stellarator)*.
This then made me think:

> *Is there some way we can modify the magneto-frictional equations such that we retrieve non-trivial equilibria that necessarily satisfy $\text{curl}[\mathbf{j} \times \mathbf{B}] = \mathbf{0}$?*

## **Modifying magnetic relaxation**

This is simple!
Recall the magneto-frictional equations *((1.4) from the Parker problem paper, dropping $\tau$'s)*:
$$
\dot{\mathbf{B}} + \text{curl}\,\mathbf{E} = \mathbf{0},  \qquad
\mathbf{E} + \mathbf{u} \times \mathbf{B} = \mathbf{0},  \qquad
\mathbf{j} = \text{curl}\,\mathbf{B},  \qquad
\mathbf{u} = \mathbf{j} \times \mathbf{B}.
$$
We just modify the second equation (1.4b) to
$$
\mathbf{E} + (\text{curl}^2\,\mathbf{u}) \times \mathbf{B} = \mathbf{0}.
$$
For the same energy $\mathcal{E} = \tfrac{1}{2}\|\mathbf{B}\|^2$ and helicity $\mathcal{H} = \tfrac{1}{2}(\mathbf{A}, \mathbf{B})$, we then have the evolution equations
$$
\dot{\mathcal{E}} = - \|\text{curl}\,\mathbf{u}\|^2 = - \|\text{curl}[\mathbf{j} \times \mathbf{B}]\|^2 \le 0,  \qquad
\dot{\mathcal{H}} = 0,
$$
alongside the preserved Arnold inequality
$$
\mathcal{E} \gtrsim |\mathcal{H}|.
$$
Thus if $\mathcal{H}|_{t=0} \ne 0$, this must relax to a non-trivial equilibrium satifsying $\text{curl}[\mathbf{j} \times \mathbf{B}] = \mathbf{0}$.

> *So can we preserve this discretely too?*

## **Structure preservation**

Sure we can!
It's a simple modification of the scheme (3.1).
We're going to introduce a new field $\omega_h \in H^h_0(\text{curl})$, a discrete approximation to $\text{curl}\,\mathbf{u}_h \approx \text{curl}\,[\mathbf{j}_h \times \mathbf{H}_h] \approx \text{curl}\,[\mathbf{j}_h \times \mathbf{B}_h]$:
> Find $\omega_h \in H^h_0(\text{curl})$ such that
> $$
> (\omega_h, \chi_h) = (\mathbf{j}_h \times \mathbf{H}_h, \text{curl}\,\chi_h)
> $$
> for all $\chi_h \in H^h_0(\text{curl})$.

We then modify (3.1b) to state:
> $$
> (\mathbf{E}_h, \mathbf{F}_h) + (\text{curl}\,\omega_h \times \mathbf{H}_h, \mathbf{F}_h) = 0
> $$
> for all $\mathbf{F}_h \in H^h_0(\text{curl})$.

Through exactly the same kind of arguments as in the manuscript, we can then prove the (semi-)discrete structures
$$
\dot{\mathcal{E}} = - \|\omega_h\|^2 \le 0,  \qquad
\dot{\mathcal{H}} = 0,
$$
implying the convergence to non-trivial equilibria, for which $\omega_h \,(\approx \text{curl}[\mathbf{j}_h \times \mathbf{B}_h]) = \mathbf{0}$.

## **Recovering flux coordinates**

Something that's very important for fusion is having flux coordinates $\psi$, as these label flux surfaces.
In the axisymmetric case, $\psi$ has a nice definition as the poloidal magnetic flux.
However, this definition doesn't really extend so readily to the non-axisymmetric case.

What we need is to find some kind of coordinate system, say $\tilde{\psi}$, for which $\tilde{\psi}$ is constant along flux surfaces.
Equivalently, we need $\tilde{\psi}$ to be constant along magnetic field lines.
Equivalently then, we need $\mathbf{B}\cdot\nabla\tilde{\psi} = 0$.

We have such a $\tilde{\psi}$ already!
The pressure $p$ is constant along flux surfaces.
We can see this by taking the dot product of the static force balance equation $\mathbf{j} \times \mathbf{B} = \nabla p$ against $\mathbf{B}$, to find $\mathbf{B}\cdot\nabla p = 0$.
Simple!

So, once we've finished our solve, we can just solve the following:
> Find $p \in H^{1,h}_0$ such that
> $$
> (\mathbf{j} \times \mathbf{B}, \nabla q) = (\nabla p, \nabla q)
> $$
> for all $q \in H^{1,h}_0$.

This aligns with the assumption in Grad–Shafranov that $p$ is a function of $\psi$.
I can't guarantee that the map from flux surfaces to $p$ will be injective—*there may be multiple surfaces with the same pressure*—but I reckon in practice this wouldn't ever really cause issue.
My assumption would be that the flux surface to pressure map is likely monotone, i.e. I don't expect it to go up and down and up and down etc. as you travel into the centre of the plasma.

## **Time discretisation**

Naturally, any Gauss method will preserve these structures to the fully discrete level.
Since we only care about the long term behaviour, I don't know if there's much reason to go to any higher order than implicit midpoint.

On a sidenote here, I wonder whether one could do some sort of splitting, using implicit Euler terms in the some places and implicit midpoint everywhere else, to accelerate the convergence?
The goal would be to accelerate the dissipation of energy *(since implicit Euler famously has extra dissipation over implicit midpoint)* without damaging the conservation of helicity.

## **Boundary conditions**

I'm not sure what the physically natural boundary conditions for such a non-axisymmetric Grad–Shafranov problem would be.
In the definition of the above scheme, we've assumed the following:
$$
\mathbf{B}_h\cdot\mathbf{n} = 0, \qquad
\mathbf{E}_h\times\mathbf{n} = \mathbf{0}, \qquad
\mathbf{j}_h\times\mathbf{n} = \mathbf{0}, \qquad
\mathbf{H}_h\times\mathbf{n} \; (\approx \mathbf{B}_h \times \mathbf{n}) = \mathbf{0}, \qquad
\omega_h\times\mathbf{n} = \mathbf{0}. \qquad
$$
Do these make sense?
This is more of a physics question really, but we can give each of these some thought...

- I would presume $\mathbf{B}\cdot\mathbf{n} = 0$ is logical, since otherwise you're going to have your flux fields trying to pass through the stellarator walls.
- The boundary condition $\mathbf{E}_h\times\mathbf{n} = \mathbf{0}$ then comes naturally from $\dot{\mathbf{B}} = - \,\text{curl}\,\mathbf{E}$.
- For the tangential current, I can't physically justify $\mathbf{j}_h\times\mathbf{n} = \mathbf{0}$.
- I think $\mathbf{H}_h\times\mathbf{n} \; (\approx \mathbf{B}_h \times \mathbf{n}) = \mathbf{0}$ is physical, as this would correspond to having zero current flux $\mathbf{j}\cdot\mathbf{n} = 0$ *(since $\mathbf{j} = \text{curl}\,\mathbf{B} = 0$)*.
- Since in equilibrium the field should satisfy $\omega = \text{curl}[\mathbf{j} \times \mathbf{B}] = \mathbf{0}$, any zero boundary conidtion on $\omega$ seems justified, including $\omega_h\times\mathbf{n} = \mathbf{0}$.

## **Euler**

Just a final little note:
this *"static force balance"* equation looks exactly like that of steady states for the Euler equation,
$$
\mathbf{u} \times \omega = \nabla p,
$$
if you simply substitute $\mathbf{B} \mapsto \mathbf{u}$.
These ideas could well be used to similarly find stationary states of the Euler equations, I believe.

---

## **A simpler form for $\mathbf{u}$**

I just realised I'm probably making this way too hard on myself;
the definition for $\mathbf{u}$ to give the desired dissipation properties could be so much simpler!
Again, recall the magneto-frictional equations:
$$
\dot{\mathbf{B}} + \text{curl}\,\mathbf{E} = \mathbf{0},  \qquad
\mathbf{E} + \mathbf{u} \times \mathbf{B} = \mathbf{0},  \qquad
\mathbf{j} = \text{curl}\,\mathbf{B},  \qquad
\mathbf{u} = \mathbf{j} \times \mathbf{B}.
$$
Instead of modifying the second equation to
$$
\mathbf{E} + (\text{curl}^2\,\mathbf{u}) \times \mathbf{B} = \mathbf{0},
$$
We're going to modify the final equation to
$$
\mathbf{u} = \mathbf{j} \times \mathbf{B} - \nabla p,  \qquad
0  =  \text{div}\,\mathbf{u}.
$$
For the same energy $\mathcal{E} = \tfrac{1}{2}\|\mathbf{B}\|^2$ and helicity $\mathcal{H} = \tfrac{1}{2}(\mathbf{A}, \mathbf{B})$ then, this system has the evolution equations
$$
\dot{\mathcal{E}} = - \|\mathbf{u}\|^2 = - \|\mathbf{j} \times \mathbf{B} - \nabla p\|^2 \le 0,  \qquad
\dot{\mathcal{H}} = 0,
$$
alongside the (as ever) preserved Arnold inequality
$$
\mathcal{E} \gtrsim |\mathcal{H}|.
$$
To go through this in excessive detail haha, the proof of energy dissipation $\dot{\mathcal{E}} = - \|\mathbf{u}\|^2$ would be as follows:
$$
\begin{aligned}
\dot{\mathcal{E}}
= \int \mathbf{B} \cdot \dot{\mathbf{B}}
= - &\int \mathbf{B} \cdot \text{curl}\,\mathbf{E}
= - \int \text{curl}\,\mathbf{B} \cdot \mathbf{E}
= - \int \mathbf{j} \cdot \mathbf{E}  \\
&= \int \mathbf{j} \cdot (\mathbf{u} \times \mathbf{B})
= - \int \mathbf{u} \cdot (\mathbf{j} \times \mathbf{B})
= - \int \mathbf{u} \cdot (\mathbf{u} - \nabla p)
= - \int \|\mathbf{u}\|^2 + \text{div}\,\mathbf{u}\,p
= - \int \|\mathbf{u}\|^2
\end{aligned}
$$
So if $\mathcal{H}|_{t=0} \ne 0$, this must relax to a non-trivial equilibrium satifsying $\mathbf{j} \times \mathbf{B} = \nabla p$.

This form is closer to the original magneto-frictional system, and moreover closer to the actual MHD equations!

## **Structure preservation**

Again, preserving this form of energy dissipation is a simple modification to the magnetic relaxation scheme from the paper.
Now, however, the options for how this is done align much more with that we're familiar with from the original Parker problem.

### ***5-field scheme***

The original Parker problem paper proposes a 4-field discretisation, in $\mathbf{B}_h$, $\mathbf{E}_h$, $\mathbf{j}_h$, $\mathbf{H}_h$.
So now, since we're looking for $\mathbf{j} \times \mathbf{B} = \nabla p$ fields instead of $\mathbf{j} \times \mathbf{B} = \mathbf{0}$ fields, surprise surprise we need at least a 5-field discretisation, in $\mathbf{B}_h$, $\mathbf{E}_h$, $\mathbf{j}_h$, $\mathbf{H}_h$, $p_h$.

Let's take $p_h \in H^{1,h}_0$.
We can first acknowledge the change in the definition of $\mathbf{u}$, by switching out
> $$
> (\mathbf{E}_h, \mathbf{F}_h) + ((\mathbf{j}_h \times \mathbf{H}_h) \times \mathbf{H}_h, \mathbf{F}_h) = 0
> $$
> for all $\mathbf{F}_h \in H^h_0(\text{curl})$.

for
> $$
> (\mathbf{E}_h, \mathbf{F}_h) + ((\mathbf{j}_h \times \mathbf{H}_h - \nabla p_h) \times \mathbf{H}_h, \mathbf{F}_h) = 0
> $$
> for all $\mathbf{F}_h \in H^h_0(\text{curl})$.

We then need to acknowledge the div-free condition on $\mathbf{u} = \mathbf{j} \times \mathbf{B} - \nabla p$, with we'll do weakly with a new equation:
> $$
> (\mathbf{j}_h \times \mathbf{H}_h, \nabla q_h) = (\nabla p_h, \nabla q_h)
> $$
> for all $q_h \in H^{1,h}_0$.

Again, through similar arguments as in the manuscript, we can then prove the (semi-)discrete structures
$$
\dot{\mathcal{E}} = - \, \|\mathbf{j}_h \times \mathbf{H}_h - \nabla p_h\|^2 \le 0,  \qquad
\dot{\mathcal{H}} = 0,
$$
implying the convergence to non-trivial equilibria, for which $\mathbf{j}_h \times \mathbf{H}_h = \nabla p_h$.

*...*

*Now...*

*...*

*I have some serious reservations about this (analogous to reservations I had about the scheme in Parker problem paper a while back)...*

*...*

### ***Some reservations...***

We know that, *provided the solver doesn't break*, we must converge to a steady state satisfying the condition $\mathbf{j}_h \times \mathbf{H}_h = \nabla p_h$.
This can be interpreted as a *pointwise, nonlinear* constraint on the magnetic field $\mathbf{B}_h$ (since $\mathbf{j}_h$, $\mathbf{H}_h$ are just functions of $\mathbf{B}_h$).

In general, say I'm considering any pointwise, nonlinear constraint on some field.
Even if there may exist an infinite continuum of continuous solutions satisfying this arbitrary constraint, the existence of such discrete solutions is very much not guaranteed:

- Consider, for example, the constraint $\|\mathbf{w}\|^2 = 1$ on some vector field $\mathbf{w}$.
Naturally there exists an infinite-dimensional continuum of such fields! :)
- Consider instead the constraint $\|\mathbf{w}_h\|^2 = 1$ on some CG vector field $\mathbf{w}_h$.
No matter how fine my mesh is, the only field satisfying such constraints is the constant field! :(

This is why pointwise, nonlinear constraints scare me.
How do we know such a discrete equilibrium exists?
Conclusion: We don't.

The only resolution to this is that at some point the solver must break.
At some point a solution will no longer exist.
We have no well-posed-ness proof for this scheme.
(Heck, we don't even know if the magneto-frictional equations are well-posed.)

*This may, for some divine reason I do not understand, be fine.*
Indeed, we've not really seen issues to date.
However, I see no reason to challenge fate if we don't need to...


### ***6-field scheme***

Just like the fix for this in the original magnetic relaxaiton model was to introduce a new field for $\mathbf{u}$, the solution is just the same here.
We're going to instead introduce two new fields $({\mathbf u}_h, p_h) \in H^h_0(\text{curl}) \times H^1_0$, giving a 6-field discretisation, in $\mathbf{B}_h$, $\mathbf{E}_h$, $\mathbf{j}_h$, $\mathbf{H}_h$, $\mathbf{u}_h$, $p_h$.

The new fields $({\mathbf u}_h, p_h) \in H^h_0(\text{curl}) \times H^1_0$ shall be defined to satisfy the new relations:
> $$
> (\mathbf{u}_h, \mathbf{v}_h) = (\mathbf{j}_h \times \mathbf{H}_h, \mathbf{v}_h) - (\nabla p_h, \mathbf{v}_h),  \qquad
> 0  =  -\,(\mathbf{u}_h, \nabla q_h),
> $$
> for all $({\mathbf v}_h, q_h) \in H^h_0(\text{curl}) \times H^1_0$.

We then modify the second equation to state:
> $$
> (\mathbf{E}_h, \mathbf{F}_h) + (\mathbf{u}_h \times \mathbf{H}_h, \mathbf{F}_h) = 0
> $$
> for all $\mathbf{F}_h \in H^h_0(\text{curl})$.

Again throught the same arguments, we can then prove the (semi-)discrete structures
$$
\dot{\mathcal{E}} = - \|\mathbf{u}_h\|^2 \le 0,  \qquad
\dot{\mathcal{H}} = 0,
$$
implying the convergence to non-trivial equilibria, for which $\mathbf{u}_h \,(\approx \mathbf{j}_h \times \mathbf{B}_h - \nabla p_h) = \mathbf{0}$.

This constraint is linear! :)
No fears from me there.
Much less scary.

## **Recovering flux coordinates**

No need to stress here.
We've already got $p_h$!

---

## **Lie advection and topology preservation**

Something important to highlight:
The equation $\dot{\mathbf{B}}  =  - \, \text{curl}\,[\mathbf{u}\times\mathbf{B}]$ is what's called a *Lie advection* equation on $\mathbf{B}$ (no matter what $\mathbf{u}$ is).
Don't stress about what *Lie advection* means too much;
the important consequence for us is that such equations *preserve the topology of the magnetic field exactly*.

Think of it like stirring around noodles in a pot.
The noodles might spread out, move around etc., but the topology can't change, i.e. if my initial noodles have a knot in them, that knot can't untie no matter how much I shake up my bowl.
Lie advection equations, including magnetic relaxation as we're interested in, have the same topology-preserving properties:
The *entire topology* of the initial magnetic field is preserved.

I just want to highlight this specifically.
Topology preservation is more than just the preservation of $\mathcal{H}$.
It's a whole continuum of topological structure, of which the helicity is just one dimension.
Conserving $\mathcal{H}$ is *nowhere near sufficient* to guarantee the preservation of the full topology.

As of yet, we don't know how well our magnetic relaxation scheme is fully *topology-preserving*.
My gut feeling is that exactly preserving the topology on each timestep would be impossible;
that would be like trying to preserve infinitely many invariants!
But the magnetic relaxation scheme is so intricately connected with exterior calculus and the discrete complex, that there may yet be some kind of *weak* topology preservation property it satisfies that we don't yet understand.
I'd have no idea how to state such a result, never mind prove it.
But it may be there.

> *We need to run some tests with topologically non-trivial initial data with zero $\mathcal{H}$.*

## **Magnetic islands**

The reason I want to highlight this is the discussion on *magnetic islands*.
These magnetic islands are a topological structure of the magnetic field.
This means that, under exact magnetic relaxation, they can neither appear nor vanish.

For this reason, I'm a little wary of any conclusions about the existence/non-existence of magnetic islands that comes from a magnetic relaxation approach.
If you have (by some divine intervention) a fully topology-preserving integrator, the existence of magnetic islands in your final field is just a function of whether you put them in your initial data.

## **What actually is the Grad–Shafranov equation?**

For me, there's an underlying question behind all this:

> *What actually is the Grad–Shafranov equation?*

The static force balance equation,
$$
\mathbf{j} \times \mathbf{B} = (\text{curl}\,\mathbf{B}) \times \mathbf{B} = \nabla p,
$$
just does not seem well-posed to me.
It's of *odd-order* in $\mathbf{B}$?
What kind of boundary conditions are appropriate for such a system?
We certainly can't impose $\mathbf{B}$ all over the boundary.

The way to make sense of such systems, I believe, is as a constrained optimisation problem;
then the questions of well-posed-ness become much less non-sensical.
For example, once might consider the question:

> *Minimise $\mathcal{E} = \tfrac{1}{2}\|\mathbf{B}\|^2$, subject to the existence of $p$ such that $(\text{curl}\,\mathbf{B}) \times \mathbf{B} = \nabla p$.*

That's well-posed!
Although the answer would just be $\mathbf{B} = \mathbf{0}$, so it's not very interesting.
One might consider:

> *For a given helicity $\mathcal{H} = \tfrac{1}{2}(\mathbf{A}, \mathbf{B})$, minimise $\mathcal{E} = \tfrac{1}{2}\|\mathbf{B}\|^2$, subject to the existence of $p$ such that $(\text{curl}\,\mathbf{B}) \times \mathbf{B} = \nabla p$*.

If we find our discrete magnetic relaxation scheme does not have any kind of weak topoology-preserving properties, my theory is that this is what we're doing when we do our *discrete* magnetic relaxation scheme, at least (i) after the above modification for the definition of $\mathbf{u}$, and (ii) assuming local minima are valid.
One might also consider:

> *For a given magnetic field topology, minimise $\mathcal{E} = \tfrac{1}{2}\|\mathbf{B}\|^2$, subject to the existence of $p$ such that $(\text{curl}\,\mathbf{B}) \times \mathbf{B} = \nabla p$*.

My theory is that this is the field to which the *continuous* magnetic relaxation equation converges.
It's not clear to me this is well-posed---*for a given topology, does there necessarily exist a $\mathbf{B}$ such that $(\text{curl}\,\mathbf{B}) \times \mathbf{B} = \nabla p$?*---but that leads into the question of well-posedness of magnetic relaxation and the Parker conjecture;
naturally these are all unsolved questions!

I guess I'm not the first person to consider these issues.
This feels like part of the course in studying Grad–Shafranov, in particular in the non-axisymmetric setting.